# ParetoBench Container Objects
Some objects are included in ParetoBench to enable users to manipulate and save data related to multi-objective optimizations. We use this notebook to explain some of their usage.

In [1]:
import os
import paretobench as pb
import tempfile

### `Population` Objects
The basic building block of optimization data is a `Population` which is supposed to represent the population of individuals in a single generation of a genetic algorithm. The populations contain the decision variables and the values of the objectives and constraints. All of these are 2D numpy arrays with the first dimension being the batch dimension and having a length equal to the population size. This is true even in the case of not having any constraints (for instance) where the second dimension is set to length zero. It also contains a place to hold the number of function evaluations performed up until this point for use in analyzing performance.

In [2]:
# Create an example population
pop = pb.Population.from_random(
    n_objectives=2, n_decision_vars=10, n_constraints=0, pop_size=32
)
print(pop)

# Examine some of the parameters
print(f"Decision variables: {pop.x.shape}")
print(f"Objectives: {pop.f.shape}")
print(f"Constraints: {pop.g.shape}")
print(f"Function evaluations: {pop.fevals}")

Population(size=32, vars=10, objs=2, cons=0, fevals=0)
Decision variables: (32, 10)
Objectives: (32, 2)
Constraints: (32, 0)
Function evaluations: 0


Population objects can also store names associated with the decision variables, objectives, and constraints. This is useful for practical optimization problems.

In [3]:
# Set naems of the decision variables, objectives, and constraints
pop.names_x = [f"Decision var {idx}" for idx in range(pop.x.shape[1])]
pop.names_f = ["Objective 1", "Objective 2"]
pop.names_g = []

### `History` Objects
Multiple populations are combined together into a `History` object that represents reports of the population of an optimization algorithm in the course of solving a problem. Each report will typically be a single generation within the genetic algorithm. The history object also holds a location for the name of the problem which is being solved.

In [4]:
# Create an example history object
hist = pb.History.from_random(
    n_populations=15, n_objectives=2, n_decision_vars=10, n_constraints=0, pop_size=32
)
hist.problem = "WFG1 (n=10)"
print(hist)

# Print some of the properties
print(f"Problem: {hist.problem}")
print(f"Number of reports: {len(hist.reports)}")

History(problem='WFG1 (n=10)', reports=15, vars=10, objs=2, cons=0)
Problem: WFG1 (n=10)
Number of reports: 15


### `Experiment` Objects
Finally, everything comes together in the `Experiment` object which represents a user performing several optimizations on a benchmark problem for one algorithm and one set of hyperparameters for that algorithm. These objects are saveable in an HDF5 file and also contain useful metadata for describing the experiment and how it was made.

In [ ]:
# Create some random history objects to store
runs = []
for _ in range(32):
    runs.append(
        pb.History.from_random(
            n_populations=15,
            n_objectives=2,
            n_decision_vars=10,
            n_constraints=0,
            pop_size=32,
        )
    )

# Create an example experiment
exp = pb.Experiment(
    runs=runs,  # The history objects in this experiment
    name="NSGA-II (default params)",  # A name for this experiment (for example, the algorithm and parameters used)
    author="The author of ParetoBench",  # Who created this
    software="ParetoBench example notebook",  # What software created this object
    software_version="1.0.0",
    comment="An example of an experiment object",
)

# Let's save the object to disk. We will use a temp. directory to not pollute your computer :)
with tempfile.TemporaryDirectory() as dir:
    # The filename we'll use
    fname = os.path.join(dir, "my_experiment.h5")

    # Save to disk
    exp.save(fname)

    # Load it back in
    exp_loaded = pb.Experiment.load(fname)

# Check out the loaded object
print(exp_loaded)
print(f"Number of histories: {len(exp_loaded.runs)}")
print(f"Name: {exp_loaded.name}")
print(
    f"Creation time: {exp_loaded.creation_time}"
)  # A UTC timestamp is added on object creation

Experiment(name='NSGA-II (default params)', created='2024-12-10', author='The author of ParetoBench', software='ParetoBench example notebook 1.0.0', runs=32)
Number of histories: 32
Name: NSGA-II (default params)
Creation time: 2024-12-10 05:21:51.993640+00:00
